<H1> InfiniBox CSI driver demo </H1>
Contact: Gregory Touretsky, @gregnsk

<H2> Install the driver using OpenShift Operator </H2>
<img src=https://github.com/gregnsk/infinibox_csi_driver_demo/blob/master/openshift-operator.png?raw=true width=50%>

<B>Notes</B>
<li>Infinidat CSI driver is supported starting from Kubernetes 1.14 (OpenShift 4.2)
<li>For earlier versions, customers may use Infinidat Dynamic Provisioner for Kubernetes (supported since Kubernetes 1.6)
<li>Kubernetes versions prior to 1.17 (OpenShift 4.4): some of CSI features like snapshots, clones, or raw block volumes might not be supported or enabled by default. Refer to relevant documentation for more details on feature-gate enablement
<li>Examples in this document use kubectl command,same commands will work with OpenShift "oc" interface
<li>Infinidat CSI driver can be installed using Helm Chart or OpenShift Operator

In [1]:
kubectl get nodes

NAME          STATUS   ROLES    AGE   VERSION
gtouret-k51   Ready    master   31d   v1.18.0
gtouret-k52   Ready    <none>   31d   v1.18.0
io307         Ready    <none>   31d   v1.18.0


In [2]:
# We recommend to install CSI driver in a dedicated namespace. Here it's running in a namespace "ibox"
# There is a single Controller (csi-infinibox-driver) instance per cluster 
# and a single Node (csi-infinibox-node) instance per worker node 
kubectl get pod -n ibox -o wide

NAME                       READY   STATUS    RESTARTS   AGE   IP              NODE          NOMINATED NODE   READINESS GATES
csi-infinibox-driver-0     5/5     Running   0          12d   10.244.2.55     io307         <none>           <none>
csi-infinibox-node-85g4t   2/2     Running   0          12d   172.20.87.214   gtouret-k51   <none>           <none>
csi-infinibox-node-jw9hx   2/2     Running   0          12d   172.20.87.99    gtouret-k52   <none>           <none>
csi-infinibox-node-qspr4   2/2     Running   0          12d   172.20.78.63    io307         <none>           <none>


<H2>Create a storage class <H2>

In [1]:
cat nfs/storageclass.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: ibox-storageclass-demo
provisioner: infinibox-csi-driver
reclaimPolicy: Delete
volumeBindingMode: Immediate
allowVolumeExpansion: true
parameters:
    pool_name: k8s_csi
    network_space: nas1
    provision_type: THIN    
    storage_protocol: nfs   
    nfs_mount_options: hard,rsize=1048576,wsize=1048576
    nfs_export_permissions : "[{'access':'RW','client':'*','no_root_squash':true}]"
    ssd_enabled: "true"
    csi.storage.k8s.io/provisioner-secret-name: infinibox-creds
    csi.storage.k8s.io/provisioner-secret-namespace: ibox
    csi.storage.k8s.io/controller-publish-secret-name: infinibox-creds
    csi.storage.k8s.io/controller-publish-secret-namespace: ibox
    csi.storage.k8s.io/node-stage-secret-name: infinibox-creds
    csi.storage.k8s.io/node-stage-secret-namespace: ibox
    csi.storage.k8s.io/node-publish-secret-name: infinibox-creds
    csi.storage.k8s.io/node-publish-secret-namespace: ibox
    csi.storage

In [2]:
kubectl create -f nfs/storageclass.yaml

storageclass.storage.k8s.io/ibox-storageclass-demo created


In [3]:
kubectl get sc ibox-storageclass-demo

NAME                     PROVISIONER            RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ibox-storageclass-demo   infinibox-csi-driver   Delete          Immediate           true                   4s


<H2> Create a Persistent Volume Claim (PVC) </H2>

In [4]:
cat nfs/pvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ibox-pvc-demo
  namespace: demo
spec:
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 1Gi
  storageClassName: ibox-storageclass-demo


In [5]:
kubectl create -f nfs/pvc.yaml

persistentvolumeclaim/ibox-pvc-demo created


In [6]:
kubectl get pvc ibox-pvc-demo -n demo

NAME            STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS             AGE
ibox-pvc-demo   Bound    csi-5c126d9990   1Gi        RWX            ibox-storageclass-demo   3s


In [7]:
kubectl get pv csi-5c126d9990

NAME             CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                STORAGECLASS             REASON   AGE
csi-5c126d9990   1Gi        RWX            Delete           Bound    demo/ibox-pvc-demo   ibox-storageclass-demo            14s


<H2> Create a snapshot class </H2>

In [8]:
cat nfs/snapshotclass.yaml

apiVersion: snapshot.storage.k8s.io/v1alpha1
kind: VolumeSnapshotClass
metadata:
  name: ibox-snapshotclass-demo
snapshotter: infinibox-csi-driver
parameters:
  csi.storage.k8s.io/snapshotter-secret-name: infinibox-creds
  csi.storage.k8s.io/snapshotter-secret-namespace: ibox



In [9]:
kubectl create -f nfs/snapshotclass.yaml

volumesnapshotclass.snapshot.storage.k8s.io/ibox-snapshotclass-demo created


In [10]:
kubectl get volumesnapshotclass

NAME                      AGE
ibox-snapshotclass-demo   3s


<H2> Create a snapshot </H2>

In [11]:
cat nfs/snapshot.yaml

apiVersion: snapshot.storage.k8s.io/v1alpha1
kind: VolumeSnapshot
metadata:
  name: ibox-pvc-snapshot-demo
  namespace: demo
spec:
  snapshotClassName: ibox-snapshotclass-demo
  source:
    name: ibox-pvc-demo
    kind: PersistentVolumeClaim



In [12]:
kubectl create -f nfs/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io/ibox-pvc-snapshot-demo created


In [13]:
kubectl get volumesnapshot ibox-pvc-snapshot-demo -n demo

NAME                     AGE
ibox-pvc-snapshot-demo   3s


In [14]:
kubectl describe volumesnapshot ibox-pvc-snapshot-demo  -n demo | grep snapcontent

  Snapshot Content Name:  snapcontent-53237985-7221-4db7-89d2-a26fe8648423


In [16]:
kubectl get volumesnapshotcontent snapcontent-53237985-7221-4db7-89d2-a26fe8648423

NAME                                               AGE
snapcontent-53237985-7221-4db7-89d2-a26fe8648423   21s


<H2> Create a new PVC from the snapshot (restore) </H2>

In [17]:
cat nfs/restoresnapshot.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ibox-snapshot-pvc-restore-demo-2
  namespace: demo
spec:
  storageClassName: ibox-storageclass-demo
  dataSource:
    name: ibox-pvc-snapshot-demo
    kind: VolumeSnapshot
    apiGroup: "snapshot.storage.k8s.io"
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Gi



In [18]:
kubectl create -f nfs/restoresnapshot.yaml

persistentvolumeclaim/ibox-snapshot-pvc-restore-demo-2 created


In [19]:
kubectl get pvc ibox-snapshot-pvc-restore-demo-2 -n demo

NAME                               STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS             AGE
ibox-snapshot-pvc-restore-demo-2   Bound    csi-e65dbf6d86   1Gi        RWX            ibox-storageclass-demo   3s


<H2> Create a new PVC from the existing PVC (clone) </H2>

In [20]:
cat nfs/clonepvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
    name: ibox-pvc-clone-demo
    namespace: demo
spec:
  accessModes:
  - ReadWriteOnce
  storageClassName: ibox-storageclass-demo
  resources:
    requests:
      storage: 1Gi
  dataSource:
    kind: PersistentVolumeClaim
    name: ibox-pvc-demo


In [21]:
kubectl create -f nfs/clonepvc.yaml

persistentvolumeclaim/ibox-pvc-clone-demo created


In [22]:
kubectl get pvc ibox-pvc-clone-demo -n demo

NAME                  STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS             AGE
ibox-pvc-clone-demo   Bound    csi-5f3edabbfa   1Gi        RWO            ibox-storageclass-demo   4s


<H2> Create a pod with a PVC attached </H2>

In [23]:
cat nfs/app.yaml

kind: Pod
apiVersion: v1
metadata:
  name: ibox-pod-pvc-demo
  namespace: demo
spec:
  containers:
    - name: my-frontend
      image: busybox
      volumeMounts:
      - mountPath: "/tmp/data"
        name: ibox-csi-volume
      command: [ "sleep", "1000" ]
  volumes:
    - name: ibox-csi-volume
      persistentVolumeClaim:
        claimName: ibox-pvc-demo


In [24]:
kubectl create -f nfs/app.yaml

pod/ibox-pod-pvc-demo created


In [27]:
kubectl get pod ibox-pod-pvc-demo -n demo

NAME                READY   STATUS    RESTARTS   AGE
ibox-pod-pvc-demo   1/1     Running   0          16s


In [28]:
kubectl exec -it ibox-pod-pvc-demo -n demo -- df -h /tmp/data

Filesystem                Size      Used Available Use% Mounted on
172.20.37.53:/csi-5c126d9990
                          1.0G      1.0M   1023.0M   0% /tmp/data


<H2>Deploying mySQL server with Helm using InfiniBox CSI driver for iSCSI</H2>

In [37]:
cat iscsi/storageclass.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: ibox-iscsi-storageclass-demo
provisioner: infinibox-csi-driver
reclaimPolicy: Delete
volumeBindingMode: Immediate
allowVolumeExpansion: true
parameters:
  csi.storage.k8s.io/provisioner-secret-name: infinibox-creds
  csi.storage.k8s.io/provisioner-secret-namespace: ibox
  csi.storage.k8s.io/controller-publish-secret-name: infinibox-creds
  csi.storage.k8s.io/controller-publish-secret-namespace: ibox
  csi.storage.k8s.io/node-stage-secret-name: infinibox-creds
  csi.storage.k8s.io/node-stage-secret-namespace: ibox
  csi.storage.k8s.io/node-publish-secret-name: infinibox-creds
  csi.storage.k8s.io/node-publish-secret-namespace: ibox
  csi.storage.k8s.io/controller-expand-secret-name: infinibox-creds
  csi.storage.k8s.io/controller-expand-secret-namespace: ibox
  useCHAP: "mutual_chap" # none / chap / mutual_chap
  fstype: ext4
  pool_name: "k8s_csi"
  network_space: "iscsi1"
  provision_type: "THIN"
  storage_protocol: "i

In [38]:
kubectl create -f iscsi/storageclass.yaml

storageclass.storage.k8s.io/ibox-iscsi-storageclass-demo created


In [43]:
helm install mysqldemo --set persistence.storageClass=ibox-iscsi-storageclass-demo,nodeSelector.protocol=iscsi stable/mysql

NAME: mysqldemo
LAST DEPLOYED: Tue May  5 11:37:41 2020
NAMESPACE: default
STATUS: deployed
REVISION: 1
NOTES:
MySQL can be accessed via port 3306 on the following DNS name from within your cluster:
mysqldemo.default.svc.cluster.local

To get your root password run:

    MYSQL_ROOT_PASSWORD=$(kubectl get secret --namespace default mysqldemo -o jsonpath="{.data.mysql-root-password}" | base64 --decode; echo)

To connect to your database:

1. Run an Ubuntu pod that you can use as a client:

    kubectl run -i --tty ubuntu --image=ubuntu:16.04 --restart=Never -- bash -il

2. Install the mysql client:

    $ apt-get update && apt-get install mysql-client -y

3. Connect using the mysql cli, then provide your password:
    $ mysql -h mysqldemo -p

To connect to your database directly from outside the K8s cluster:
    MYSQL_HOST=127.0.0.1
    MYSQL_PORT=3306

    # Execute the following command to route the connection:
    kubectl port-forward svc/mysqldemo 3306

    mysql -h ${MYSQL_HOST} -P$

In [44]:
kubectl get deployment mysqldemo

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
mysqldemo   1/1     1            1           32s


In [48]:
kubectl get pvc mysqldemo

NAME        STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS                   AGE
mysqldemo   Bound    csi-12e5e6a266   8Gi        RWO            ibox-iscsi-storageclass-demo   95s


In [51]:
POD=`kubectl get pod | grep mysqldemo | awk '{print $1}'`
kubectl exec -it $POD -- df /var/lib/mysql

Filesystem          1K-blocks   Used Available Use% Mounted on
/dev/mapper/mpathdn   8191416 251852   7923180   4% /var/lib/mysql


<H2> Clean everything </H2>

In [52]:
helm uninstall mysqldemo

release "mysqldemo" uninstalled


In [53]:
kubectl delete -f nfs/app.yaml

pod "ibox-pod-pvc-demo" deleted


In [54]:
kubectl delete -f nfs/clonepvc.yaml

persistentvolumeclaim "ibox-pvc-clone-demo" deleted


In [55]:
kubectl delete -f nfs/restoresnapshot.yaml

persistentvolumeclaim "ibox-snapshot-pvc-restore-demo-2" deleted


In [56]:
kubectl delete -f nfs/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io "ibox-pvc-snapshot-demo" deleted


In [57]:
kubectl delete -f nfs/snapshotclass.yaml

volumesnapshotclass.snapshot.storage.k8s.io "ibox-snapshotclass-demo" deleted


In [58]:
kubectl delete -f nfs/pvc.yaml

persistentvolumeclaim "ibox-pvc-demo" deleted


In [59]:
kubectl delete -f nfs/storageclass.yaml

storageclass.storage.k8s.io "ibox-storageclass-demo" deleted


In [60]:
kubectl delete -f iscsi/storageclass.yaml

storageclass.storage.k8s.io "ibox-iscsi-storageclass-demo" deleted
